In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
import seaborn as sns

In [2]:
'''Old dataframe, deleted because it took up too much space on my computer'''
# kkbox = pd.read_csv('/Users/galvanize/Documents/projects/kkbox-churn-prediction-challenge/made_by_me/member_sample.csv', sep=',')
# kkbox.head()

# age_kkbox = kkbox.copy()

'''Made a new dataframe to change the ages of users that were outside of a normal human lifespan to be more 
reasonable. There were already multiple '0's, so changed the impossible ages into 0's. '''

# def fixed_ages(age):
# #     if age > 100 or age < 14:
# #         return 0
# #     return age
#       return 0 if age > 100 or age < 14 else age

# age_kkbox['bd'] = kkbox['bd'].apply(fixed_ages)
# age_kkbox['bd'].value_counts()



,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,...,membership_expire_date,is_cancel,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-01,11,5,2,3,31,45,8273.919
1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-19,53,18,7,2,34,100,11547.321
2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-25,17,9,3,7,31,62,9686.813
3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-05,19,10,6,3,41,66,12757.596
4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-26,41,4,4,10,44,92,13503.578


In [11]:
age_kkbox.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14943464 entries, 0 to 14943463
Data columns (total 22 columns):
msno                      14943464 non-null object
city                      14943464 non-null int64
bd                        14943464 non-null int64
gender                    8028018 non-null object
registered_via            14943464 non-null int64
registration_init_time    14943464 non-null object
payment_method_id         14943464 non-null int64
payment_plan_days         14943464 non-null int64
plan_list_price           14943464 non-null int64
actual_amount_paid        14943464 non-null int64
is_auto_renew             14943464 non-null object
transaction_date          14943464 non-null object
membership_expire_date    14943464 non-null object
is_cancel                 14943464 non-null object
date_entry                14943464 non-null object
num_25                    14943464 non-null int64
num_50                    14943464 non-null int64
num_75                    14

In [13]:
age_kkbox.head()

,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,...,membership_expire_date,is_cancel,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-01,11,5,2,3,31,45,8273.919
1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-19,53,18,7,2,34,100,11547.321
2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-25,17,9,3,7,31,62,9686.813
3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-05,19,10,6,3,41,66,12757.596
4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-04-18,f,2017-03-26,41,4,4,10,44,92,13503.578


In [14]:
'''To find total number of songs listened to by each user, on each day they listened to music'''

age_kkbox['num_total'] =  age_kkbox[['num_25', 'num_50', 'num_75', 'num_985', 'num_100']].sum(axis=1)
age_kkbox.head()

,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,...,is_cancel,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,num_total
0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-01,11,5,2,3,31,45,8273.919,52
1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-19,53,18,7,2,34,100,11547.321,114
2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-25,17,9,3,7,31,62,9686.813,67
3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-05,19,10,6,3,41,66,12757.596,79
4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-26,41,4,4,10,44,92,13503.578,103


In [53]:
cols = age_kkbox.columns.tolist()
cols

['msno',
 'city',
 'bd',
 'gender',
 'registered_via',
 'registration_init_time',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'transaction_date',
 'membership_expire_date',
 'is_cancel',
 'date_entry',
 'num_25',
 'num_50',
 'num_75',
 'num_985',
 'num_100',
 'num_unq',
 'total_secs',
 'num_total']

In [54]:
'''Rearranging the order of my columns'''

cols = cols[:-2] + cols[-1:] + cols[-2:-1]
cols

['msno',
 'city',
 'bd',
 'gender',
 'registered_via',
 'registration_init_time',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'transaction_date',
 'membership_expire_date',
 'is_cancel',
 'date_entry',
 'num_25',
 'num_50',
 'num_75',
 'num_985',
 'num_100',
 'num_unq',
 'num_total',
 'total_secs']

In [55]:
kkbox_sample = age_kkbox[cols]
kkbox_sample

,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,...,is_cancel,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,num_total,total_secs
0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-01,11,5,2,3,31,45,52,8273.919
1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-19,53,18,7,2,34,100,114,11547.321
2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-25,17,9,3,7,31,62,67,9686.813
3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-05,19,10,6,3,41,66,79,12757.596
4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,f,2017-03-26,41,4,4,10,44,92,103,13503.578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14943459,zXDu//E2cLmt6DUUtkitB2g0vpLcIc03DO8ZMgAv0hA=,1,0,NaN,7,2013-09-29,41,30,149,149,...,f,2017-03-30,0,0,1,1,3,5,5,907.411
14943460,zXDu//E2cLmt6DUUtkitB2g0vpLcIc03DO8ZMgAv0hA=,1,0,NaN,7,2013-09-29,41,30,149,149,...,f,2017-03-09,1,0,1,0,8,8,10,1912.061
14943461,zXDu//E2cLmt6DUUtkitB2g0vpLcIc03DO8ZMgAv0hA=,1,0,NaN,7,2013-09-29,41,30,149,149,...,f,2017-03-19,15,6,5,4,44,60,74,11993.144
14943462,zXDu//E2cLmt6DUUtkitB2g0vpLcIc03DO8ZMgAv0hA=,1,0,NaN,7,2013-09-29,41,30,149,149,...,f,2017-03-06,6,1,1,2,9,18,19,2568.837


In [58]:
'''Turning nulls into missing'''

kkbox_sample['gender'] = kkbox_sample['gender'].fillna('missing')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14943464 entries, 0 to 14943463
Data columns (total 23 columns):
msno                      object
city                      int64
bd                        int64
gender                    object
registered_via            int64
registration_init_time    object
payment_method_id         int64
payment_plan_days         int64
plan_list_price           int64
actual_amount_paid        int64
is_auto_renew             object
transaction_date          object
membership_expire_date    object
is_cancel                 object
date_entry                object
num_25                    int64
num_50                    int64
num_75                    int64
num_985                   int64
num_100                   int64
num_unq                   int64
num_total                 int64
total_secs                float64
dtypes: float64(1), int64(14), object(8)
memory usage: 2.6+ GB


In [59]:
kkbox_sample.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14943464 entries, 0 to 14943463
Data columns (total 23 columns):
msno                      14943464 non-null object
city                      14943464 non-null int64
bd                        14943464 non-null int64
gender                    14943464 non-null object
registered_via            14943464 non-null int64
registration_init_time    14943464 non-null object
payment_method_id         14943464 non-null int64
payment_plan_days         14943464 non-null int64
plan_list_price           14943464 non-null int64
actual_amount_paid        14943464 non-null int64
is_auto_renew             14943464 non-null object
transaction_date          14943464 non-null object
membership_expire_date    14943464 non-null object
is_cancel                 14943464 non-null object
date_entry                14943464 non-null object
num_25                    14943464 non-null int64
num_50                    14943464 non-null int64
num_75                    1

In [ ]:
'''Made a csv with fixed ages and rearranged columns'''
# kkbox_sample.to_csv(r'/Users/galvanize/Documents/projects/kkbox-churn-prediction-challenge/made_by_me/kkbox.csv')

In [3]:
kkbox = pd.read_csv('/Users/galvanize/Documents/projects/kkbox-churn-prediction-challenge/made_by_me/kkbox.csv')

In [4]:
kkbox.head()

,Unnamed: 0,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,is_cancel,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,num_total,total_secs
0,0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,...,f,2017-03-01,11,5,2,3,31,45,52,8273.919
1,1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,...,f,2017-03-19,53,18,7,2,34,100,114,11547.321
2,2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,...,f,2017-03-25,17,9,3,7,31,62,67,9686.813
3,3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,...,f,2017-03-05,19,10,6,3,41,66,79,12757.596
4,4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,...,f,2017-03-26,41,4,4,10,44,92,103,13503.578


In [67]:
'''Making a member dataframe'''
# mem_list = kkbox_sample['msno'].unique()

# mem_df = pd.DataFrame(mem_list, columns=['members'])

In [70]:
'''CSV for sampled members'''
# mem_df.to_csv(r'/Users/galvanize/Documents/projects/kkbox-churn-prediction-challenge/made_by_me/member_samp.csv')

In [5]:
'''1 = churn, 0 = renewal'''

train = pd.read_csv('/Users/galvanize/Documents/projects/kkbox-churn-prediction-challenge/train_v2.csv', sep=',')
train.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [6]:
train.is_churn.value_counts()

0    883630
1     87330
Name: is_churn, dtype: int64

In [73]:
kkbox_churn = pd.merge(left= kkbox_sample, right= train, left_on= 'msno', right_on= 'msno')
kkbox_churn.head()

,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,...,date_entry,num_25,num_50,num_75,num_985,num_100,num_unq,num_total,total_secs,is_churn
0,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-03-01,11,5,2,3,31,45,52,8273.919,0
1,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-03-19,53,18,7,2,34,100,114,11547.321,0
2,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-03-25,17,9,3,7,31,62,67,9686.813,0
3,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-03-05,19,10,6,3,41,66,79,12757.596,0
4,3xmvdgcfiPnpzVvQH2LxgiQmhupt9ml4Zwp0R2lNfGA=,4,51,male,9,2010-03-03,40,30,149,149,...,2017-03-26,41,4,4,10,44,92,103,13503.578,0


In [ ]:
# formula = 'is_churn ~ C(gender) + C(date_entry) + num_25 + num_50 + num_75 + num_985'

# fitted_model = smf.ols(formula=formula, data=kkbox_churn).fit()
# fitted_model.summary()

In [1]:
kkbox_churn.info()

NameError: name 'kkbox_churn' is not defined